In [53]:
from tqdm import tqdm_notebook, tqdm
import pandas as pd
from os.path import join as oj
import torch
import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
sys.path.append('../fit')
from cd import cd_batch_text, softmax_out, cd_penalty_for_one
import matplotlib.pyplot as plt
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import pandas
import csv

import numpy as np
np.random.seed(42)
import pickle as pkl

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [54]:
#!/usr/bin/env python
# coding: utf-8

# In[2]:


from os.path import join as oj


# In[3]:


import sys, time
sys.path.insert(1, oj(sys.path[0], '..'))  # insert parent path
sys.path.append('../fit')


# In[4]:


from cd import cd_batch_text, softmax_out, cd_penalty_for_one


# In[24]:

%matplotlib inline
%load_ext autoreload
%autoreload 2


import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext import data
import torch.optim as O
torch.manual_seed(42)
from model import LSTMSentiment


from torchtext.data import Field
import os
import pandas
import csv
from torchtext.data import TabularDataset
import torchtext
import numpy as np
#np.random.seed(42)
import torch
import pickle
from tqdm import tqdm

# In[6]:


torch.cuda.set_device(0)


# # Make dataset
# 

# In[202]:

repeats = 10 
num_epochs = 50
loss_weight_list = [1,]#0, 0.01, 0.1]#, 1, 2, 10]
num_in_train = 20
this_batch_size = 5

char_class = ['a', 'b']
char_noise = ['c', 'd', 'e', 'f']
num_data =100
dataset_path = "../data"
max_length = 9
noise = 0.0

# In[203]:


def make_dataset(num, noise = 0.0):
    dataset_list=[]
    has_noise = np.random.uniform(size = num) < noise
    for i in range(num):
        
        my_output =np.random.randint(2)
        my_input = [char_noise[np.random.randint(len(char_noise))] for x in range(max_length)] 
        if has_noise[i]:
            my_input[np.random.randint(max_length)] = char_class[1-my_output]
            my_input[np.random.randint(max_length)] = char_class[my_output]
            
        my_input[np.random.randint(max_length)] = char_class[my_output]

        dataset_list.append([' '.join(my_input),  my_output ])
    return dataset_list
def write_dataset(file_path, file_name,num,add_noise =0):
    my_dataset = make_dataset(num)
    with open(os.path.join(file_path, file_name), 'w') as csv_file:
        writer = csv.writer(csv_file)
        for line in my_dataset:
            writer.writerow(line)


# In[204]:

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [55]:
gpu_device =0

In [56]:



tokenize = lambda x: x.split()
TEXT = Field(sequential=True,tokenize = tokenize,lower=True, unk_token=None)
LABEL = Field(sequential=False, use_vocab=False,is_target  = True, unk_token=None)


# In[206]:


tv_datafields = [ ("text", TEXT), ("label", LABEL)]
train, dev,test = TabularDataset.splits(
               path=dataset_path, # the root directory where the data lies
               train='train.csv', validation="valid.csv", test = "test.csv",
               format='csv', 


               skip_header=False, # if your csv header has a header, make sure to pass this to ensure it doesn't get proceesed as data!
               fields=tv_datafields)



train_iter, dev_iter, test_iter = data.BucketIterator.splits(
    (train, dev, test), batch_size=this_batch_size, device=torch.device(gpu_device),
    sort_key=lambda x: len(x.text), # the BucketIterator needs to be told what function it should use to group the data.
             sort_within_batch=False,
    shuffle =True,
             repeat=False) # we pass repeat=False because we want to wrap this Iterator layer.,)

In [57]:
# In[211]:


TEXT.build_vocab(train, dev, test)
LABEL.build_vocab(train, dev, test)


# In[212]:


TEXT.vocab.vectors = torch.eye(len(TEXT.vocab))


# In[213]:


class my_config:
    d_hidden = 4
    n_embed =len(TEXT.vocab)
    d_embed =len(TEXT.vocab) 
    d_out =len(LABEL.vocab)
    batch_size =10


# In[252]:

In [58]:
for batch_idx, batch in enumerate(train_iter):
    pass

In [62]:
model = LSTMSentiment(my_config)
model.embed.weight.data = TEXT.vocab.vectors
model.cuda()

LSTMSentiment(
  (embed): Embedding(7, 7)
  (lstm): LSTM(7, 4)
  (hidden_to_label): Linear(in_features=4, out_features=2, bias=True)
)

In [59]:
TEXT.vocab.itos

['<pad>', 'e', 'f', 'c', 'd', 'a', 'b']

In [63]:
def is_in_relevant(batch, start, stop,  class_rules,dim =0):

    #XXX only for current model where relevant bigger five
    rel_digits = ((batch.label ==class_rules[0][0])[None, :] *(batch.text ==class_rules[0][1])) + (batch.label ==class_rules[1][0])[None, :] *(batch.text ==class_rules[1][1])
    relevant = rel_digits[start:stop].sum(dim=0)
    irrelevant = rel_digits.sum(dim=0) - relevant
    test_out = torch.cat((relevant[:, None], irrelevant[:, None]), 1)
    test_out = test_out/ test_out.sum(dim=1)[:, None]
    return test_out
    

def cd_penalty_for_one(batch, model1, start, stop,class_rules):
   # get output
    model1_output = cd_batch_text(batch, model1, start, stop)
    # only use the correct class
    correct_idx = (batch.label, torch.arange(batch.label.shape[0]))
    model1_softmax = softmax_out((model1_output[0][correct_idx],model1_output[1][correct_idx]))
    model2_softmax = is_in_relevant(batch, start, stop,class_rules).cuda().float()
    
   
    output = -(torch.log(model1_softmax)*model2_softmax).sum() 
 
    return output
cd_penalty_for_one(batch, model, 0, 1, ((1, 6), (0, 5)))

tensor(3.3591, device='cuda:0', grad_fn=<NegBackward>)

In [64]:
is_in_relevant(batch, 0, 1, ((1, 6), (0, 5)))

tensor([[0, 1],
        [0, 1],
        [0, 1],
        [1, 0],
        [0, 1]], device='cuda:0')

In [66]:
batch.text

tensor([[4, 2, 3, 6, 2],
        [6, 2, 4, 2, 5],
        [1, 2, 5, 2, 1],
        [2, 1, 4, 2, 2],
        [5, 5, 3, 3, 3]], device='cuda:0')

In [76]:
LABEL.vocab.stoi['o']

0

In [75]:
LABEL.vocab.stoi.keys()

dict_keys(['0', '1', 'o', 'ff'])

In [70]:
type(LABEL.vocab.itos[0])

str

In [7]:
results_folder = "../results"
fnames = sorted([oj(results_folder, fname) for fname in os.listdir(results_folder)]) # filenames in the directory
results_list = [pd.Series(pkl.load(open(fname, "rb"))) for fname in tqdm(fnames) ] 
results = pd.concat(results_list, axis=1).T.infer_objects() # pandas dataframe w/ hyperparams and weights stored

results["final_acc"] = [max(x) for x in results["acc_test"]]
results["min_test_loss"] = [min(x) for x in results["losses_test"]]
results["num_epochs"] = [len(x) for x in results["losses_test"]]


100%|██████████| 2/2 [00:00<00:00, 359.61it/s]


In [30]:
LABEL.vocab.stoi

defaultdict(<function torchtext.vocab._default_unk_index()>, {'0': 0, '1': 1})

In [32]:
batch.label

tensor([1, 0, 0, 0, 0], device='cuda:0')

In [31]:
batch.text

tensor([[6, 2, 1, 4, 3],
        [4, 3, 5, 5, 3],
        [3, 5, 2, 1, 1],
        [3, 2, 3, 3, 5]], device='cuda:0')

LSTMSentiment(
  (embed): Embedding(7, 7)
  (lstm): LSTM(7, 4)
  (hidden_to_label): Linear(in_features=4, out_features=2, bias=True)
)